## Pytorch

Load libraries

In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
import torchvision.transforms as transforms
from torchvision.transforms import ToTensor

CNN architecture

In [121]:
model = nn.Sequential(
    nn.Conv2d(1, 64, kernel_size=3),
    nn.BatchNorm2d(64),
    nn.ReLU(),
    nn.Conv2d(64, 128, kernel_size=3),
    nn.ReLU(),
    nn.MaxPool2d(2),
    nn.Dropout2d(0.25),
    nn.Flatten(),
    nn.Linear(128 * 12 * 12, 512),  # Increase the size of the fully connected layer
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(512, 10)
)

Dataset and data loader

In [122]:
# Set random seed for reproducibility
torch.manual_seed(42)

# Load MNIST dataset and apply transforms
train_dataset = MNIST(root='.', train=True, download=True, transform=ToTensor())
test_dataset = MNIST(root='.', train=False, download=True, transform=ToTensor())

# Define batch size
batch_size = 128

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

Model, lossfunction and optimizer

In [123]:
# Create an instance of the CNN model
# model = CNN()

# Define loss function
loss_fn = nn.CrossEntropyLoss()

# Define optimizer
optimizer = optim.Adam(model.parameters(), lr=0.0005)


Model Training

In [124]:
# Set the number of epochs
num_epochs = 15

# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Training loop
for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        
        outputs = model(images)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item() * images.size(0)
    
    # Calculate average training loss
    train_loss /= len(train_loader.dataset)
    
    # Evaluate the model on the test set
    model.eval()
    correct = 0
    total = 0
    
    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    # Calculate validation accuracy
    val_accuracy = 100 * correct / total
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%")
    
    # Break the training loop if desired accuracy is reached
    if val_accuracy >= 99.40:
        break



Epoch [1/15], Train Loss: 0.1510, Val Accuracy: 98.56%
Epoch [2/15], Train Loss: 0.0521, Val Accuracy: 98.85%
Epoch [3/15], Train Loss: 0.0368, Val Accuracy: 99.04%
Epoch [4/15], Train Loss: 0.0278, Val Accuracy: 98.77%
Epoch [5/15], Train Loss: 0.0210, Val Accuracy: 99.02%
Epoch [6/15], Train Loss: 0.0183, Val Accuracy: 99.03%
Epoch [7/15], Train Loss: 0.0161, Val Accuracy: 99.26%
Epoch [8/15], Train Loss: 0.0131, Val Accuracy: 99.22%
Epoch [9/15], Train Loss: 0.0119, Val Accuracy: 99.27%
Epoch [10/15], Train Loss: 0.0112, Val Accuracy: 99.26%
Epoch [11/15], Train Loss: 0.0101, Val Accuracy: 99.34%
Epoch [12/15], Train Loss: 0.0095, Val Accuracy: 99.21%
Epoch [13/15], Train Loss: 0.0090, Val Accuracy: 99.15%
Epoch [14/15], Train Loss: 0.0063, Val Accuracy: 99.18%
Epoch [15/15], Train Loss: 0.0078, Val Accuracy: 99.15%


## Tensorflow



In [58]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

Load and preprocess the MNIST dataset


In [59]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(-1, 28, 28, 1).astype('float32') / 255.0
x_test = x_test.reshape(-1, 28, 28, 1).astype('float32') / 255.0


Define the model


In [60]:
# model = Sequential([
#     Conv2D(16, (3, 3), activation='relu', input_shape=(28, 28, 1)),
#     MaxPooling2D((2, 2)),
#     Conv2D(32, (3, 3), activation='relu'),
#     MaxPooling2D((2, 2)),
#     Flatten(),
#     Dense(128, activation='relu'),
#     Dense(10, activation='softmax')
# ])

In [61]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),  # Adding dropout regularization
    Dense(10, activation='softmax')
])

 Compile the model

In [67]:
learning_rate = 0.00001
optimizer = Adam(learning_rate=learning_rate)


model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

Train the model

In [68]:
history = model.fit(x_train, y_train, validation_split=0.1, epochs=20, batch_size=128)

Epoch 1/20
422/422 [==============================] - 4s 5ms/step - loss: 0.0087 - accuracy: 0.9970 - val_loss: 0.0346 - val_accuracy: 0.9925
Epoch 2/20
422/422 [==============================] - 2s 5ms/step - loss: 0.0073 - accuracy: 0.9974 - val_loss: 0.0355 - val_accuracy: 0.9927
Epoch 3/20
422/422 [==============================] - 2s 5ms/step - loss: 0.0067 - accuracy: 0.9979 - val_loss: 0.0361 - val_accuracy: 0.9923
Epoch 4/20
422/422 [==============================] - 2s 4ms/step - loss: 0.0064 - accuracy: 0.9980 - val_loss: 0.0366 - val_accuracy: 0.9922
Epoch 5/20
422/422 [==============================] - 2s 5ms/step - loss: 0.0066 - accuracy: 0.9979 - val_loss: 0.0372 - val_accuracy: 0.9920
Epoch 6/20
422/422 [==============================] - 2s 4ms/step - loss: 0.0058 - accuracy: 0.9982 - val_loss: 0.0373 - val_accuracy: 0.9925
Epoch 7/20
422/422 [==============================] - 2s 5ms/step - loss: 0.0057 - accuracy: 0.9983 - val_loss: 0.0378 - val_accuracy: 0.9925
Epoch 

Evaluate the model on test set

In [69]:
test_loss, test_accuracy = model.evaluate(x_test, y_test)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

313/313 [==============================] - 1s 3ms/step - loss: 0.0267 - accuracy: 0.9945
Test Loss: 0.0267
Test Accuracy: 0.9945
